# HTTP Module

## 1. Define http server

### 1.1. Define routers map 

In [ ]:
const routers = {
    
    // GET http://localhost:8081/, index page, return json object
    ['GET /'](context) {
        let message = 'Hello node.js';
        
        // check if name argument exist
        if (context.parameters.name) {
            message = `${message}, have fun ${context.parameters.name}`;
        }
        
        return {
            type: 'json',
            headers: {
                auth: 'alvin'
            },
            content: {
                status: 'success',
                message: message
            }
        };
    },
    
    // GET http://localhost:8081/login, return html view
    ['GET /login']() {
        return {
            type: 'html',
            view: 'login',
            parameters: {
                name: 'Alvin'
            }
        }
    },
    
    // POST http://localhost:8081/login, return redirect url
    ['POST /login'](context) {
        const name = context.parameters.name;
        const password = context.parameters.password;
        if (!name || !password) {
            let errors = {};
            if (!name) {
                errors['name'] = 'name cannot be empty';
            }
            if (!password) {
                errors['password'] = 'password cannot be empty';
            }
            return {
                type: 'html',
                view: 'login',
                parameters: {
                    errors: errors,
                    name: name,
                    password: password
                }
            };
        }
        
        return {
            type: 'redirect',
            url: `/?name=${name}`
        };
    },
    
    // POST http://localhost:8081/redirect, return redirect url
    ['GET /redirect'](context) {
        let url = '/';
        if (context.parameters.url) {
            url = context.parameters.url;
        }
        return {
            type: 'redirect',
            url: url
        };
    },
    
    // 404 error
    ['404']() {
        return {
            type: 'json',
            content: {
                status: 'error',
                message: 'Resource not found.'
            }
        };
    }
};

### 1.2. Define http event callbacks

In [ ]:
const url = require('url');
const querystring = require('querystring');
const jade = require('jade');

function httpService(request, response) {
    console.log(`* request version: ${request.httpVersion}`);
    console.log(`* request header is: ${request.headers.host}`);

    // parse url and get arguments
    const uri = url.parse(request.url, true, false);
    const context = {
        headers: request.headers, 
        parameters: {
            ...uri.query
        }
    };

    // receive every chunk from request
    const chunks = [];
    request.on('data', chunk => chunks.push(chunk));
    
    // when all request data is received
    request.on('end', () => {
        if (chunks.length > 0) {
            const body = chunks.join('');
            console.log(`* request body is: ${body}`);

            context.parameters = {
                ...querystring.parse(body),
                ...context.parameters
            }
        }

        // execute 
        function execute(code, router) {
            if (router) {
                const result = (() => {
                    if (typeof(router) === 'function') {
                        return router(context);
                    }
                    return router.controller(context);
                })();
                
                switch (result.type) {
                case 'redirect':
                    response.writeHeader(302, { 'Location': result.url });
                    response.end();
                    break;
                case 'json':
                    response.writeHeader(code, {
                        'Content-Type': 'application/json',
                        ...result.headers
                    });
                    response.end(JSON.stringify(result.content));
                    break;
                case 'html':
                    response.writeHeader(code, {
                        'Content-Type': 'text/html',
                        ...result.headers
                    });
                    jade.renderFile(`view/${result.view || 'index'}.jade`, {
                        errors: {}, 
                        ...result.parameters
                    }, (err, html) => {
                        if (err) {
                            throw err;
                        }
                        response.end(html);
                    });
                    break;
                default:
                    response.writeHeader(500, { 'Content-Type': 'text/html' });
                    response.end();
                }
            } else {
                response.writeHeader(404, { 'Content-Type': 'text/html' });
                response.end();
            }
        }
        
        // find router function by request method and url path
        const router = routers[`${request.method} ${uri.pathname || '/'}`];
        try {
            if (router) {
                execute(200, router);
            } else {
                execute(404, routers['404']);
            }
        } catch (e) {
            console.log(e);
            execute(500, routers['500']);
        }
    });
}

### 1.3. Create http server

In [ ]:
const http = require('http');

// create http server instance
const server = http.createServer(httpService)
    .on('close', () => server.close());

// the max header count limit
server.maxHeadersCount = 1000;

// set socket timeout. when server is timeout, a 'timeout' event would be raise, and pass socket as argument
// set timeout value is 0 will disable event raise 
server.setTimeout(1000, socket => console.log(''));

// idle time of socket timeout
server.timeout = 120000;

// start listen on server, and callback if listen was started
server.listen(8081, '0.0.0.0', () => {
    console.log('Listen 8081 successful');
});

### 1.3. Create http server

In [ ]:
server.close();